# **Building SQL Agent using Langchain**

##**Real-World Use Cases**

“Where can you use this SQL Agent?”

Business analytics dashboards

Finance reporting

Retail inventory queries

HR employee information agents

Customer support data lookup

Learning and teaching SQL interactively

### Anywhere you want AI to analyze database data without manually writing SQL.

##**Reference Links:**

- https://docs.langchain.com/oss/python/langchain/sql-agent
- https://docs.langchain.com/oss/python/langchain/agents
- https://www.sqlitetutorial.net/sqlite-sample-database/

# **Difference in our project from Official Documentation Example**

# Our Code Uses the Newer OpenAI Tools Agent

Uses: agent_type="openai-tools"

 -- More stable

 -- Less prone to hallucinations

 -- Maintains clean SQL structure

 -- Designed for GPT-4.1 and future models

# Old Docs Used ReAct SQL Agent (official doc)

 -- Based on step-by-step reasoning prompts

 -- More error-prone

 -- Sometimes writes invalid SQL

 -- Required manual prompt engineering

##**Which Is Better?**

The OpenAI Tools approach is unquestionably better because:

- It follows strict JSON schema
- LLM cannot hallucinate random SQL keywords
- Tool calling ensures SQL generation is deterministic
- More reliable for production
- Supported long-term

In [22]:
#pip install langchain  langgraph  langchain-community

In [23]:
# downgrading the installation as some features are removed in the latest langchain version

!pip install langchain==0.3.4
!pip install langchain-community==0.3.3. # for integrations (tools, agents, SQL utilities)
!pip install langchain-openai==0.2.2 # wrapper for OpenAI models


In [24]:
!pip install langsmith # # debugging/ monitoring tools for LLM applications

In [43]:
# Set up LangSmith to inspect what is happening inside your chain or agent.
# Then set the following environment variables:


import os, getpass

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")


# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY=""

Enter your LangSmith API key: ··········


In [49]:
# lsv2_pt_989ea25d04094084a81d98343a18_1008b2be98

#**Select an LLM**

# Select a model that supports tool-calling:
# OpenAI, Anthropic, Azure, Google Gemini, AWS Bedrock


In [26]:
#pip install -U "langchain[openai]"

In [27]:
from langchain_openai import ChatOpenAI #langchain wrapper for OpenAI

In [44]:
# Import the 'os' module for environment variables and 'getpass' for secure password input
import os, getpass

# Prompt the user to securely enter their OpenAI API Key, which will not be displayed on screen
# This key is then stored as an environment variable for use by the LangChain library
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

model = ChatOpenAI(model="gpt-4.1")



Enter your OpenAI API Key: ··········


In [29]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(
#     model="gpt-4.1",
#     # stream_usage=True,
#     # temperature=None,
#     # max_tokens=None,
#     # timeout=None,
#     # reasoning_effort="low",
#     # max_retries=2,
#     # api_key="...",  # If you prefer to pass api key in directly
#     # base_url="...",
#     # organization="...",
#     # other params...
# )

# **Configure the database**

We will be creating a SQLite database for this tutorial. SQLite is a lightweight database that is easy to set up and use. We will be loading the chinook database, which is a sample database that represents a digital media store.
For convenience, we have hosted the database (Chinook.db) on a public GCS bucket.


Now let's write a script that **downloads a database file (`Chinook.db`) only if it is not already present** in your working directory.

### Step-by-step:

### **1. Import libraries**

```python
import requests, pathlib
```

* `requests` → used to download files from the internet
* `pathlib` → used to handle file paths cleanly

---

### **2. URL of the file to download**

```python
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
```

This is where the `.db` file is hosted online.

---

### **3. Define the local file path**

```python
local_path = pathlib.Path("Chinook.db")
```

This means:
“Save or check a file named **Chinook.db** in the current folder.”

---

### **4. Check if the file already exists**

```python
if local_path.exists():
    print(f"{local_path} already exists, skipping download.")
```

If the file is already downloaded → no need to download again.

---

### **5. Otherwise, download the file**

```python
response = requests.get(url)
```

Makes an HTTP GET request to fetch the file.

---

### **6. If request is successful**

```python
if response.status_code == 200:
    local_path.write_bytes(response.content)
```

* `response.status_code == 200` → download was successful
* `write_bytes` → saves the file as raw bytes

---

### **7. Error handling**

```python
else:
    print(f"Failed to download the file. Status code: {response.status_code}")
```

If download fails, print the reason.

---

# **What this achieves (Real-world use case)**

This is a **safe downloader** that avoids downloading a file multiple times.
Ideal for:

* Databases
* ML datasets
* Assets needed in notebooks (Colab, Jupyter)

Perfect for reproducible data workflows.


In [45]:
import requests, pathlib #http request, path handling

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"
local_path = pathlib.Path("Chinook.db")

if local_path.exists():
    print(f"{local_path} already exists, skipping download.")
else:
    response = requests.get(url)
    if response.status_code == 200:
        local_path.write_bytes(response.content)
        print(f"File downloaded and saved as {local_path}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

Chinook.db already exists, skipping download.


## **And after running the above script, we'll see chinook.db downloaded in our directory**

## Now, we will use a handy SQL database wrapper available in the langchain_community package to interact with the database. The wrapper provides a simple interface to execute SQL queries and fetch results.

In [46]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [32]:
print(db.run("Select * from Album"))

[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3), (6, 'Jagged Little Pill', 4), (7, 'Facelift', 5), (8, 'Warner 25 Anos', 6), (9, 'Plays Metallica By Four Cellos', 7), (10, 'Audioslave', 8), (11, 'Out Of Exile', 8), (12, 'BackBeat Soundtrack', 9), (13, 'The Best Of Billy Cobham', 10), (14, 'Alcohol Fueled Brewtality Live! [Disc 1]', 11), (15, 'Alcohol Fueled Brewtality Live! [Disc 2]', 11), (16, 'Black Sabbath', 12), (17, 'Black Sabbath Vol. 4 (Remaster)', 12), (18, 'Body Count', 13), (19, 'Chemical Wedding', 14), (20, 'The Best Of Buddy Guy - The Millenium Collection', 15), (21, 'Prenda Minha', 16), (22, 'Sozinho Remix Ao Vivo', 16), (23, 'Minha Historia', 17), (24, 'Afrociberdelia', 18), (25, 'Da Lama Ao Caos', 18), (26, 'Acústico MTV [Live]', 19), (27, 'Cidade Negra - Hits', 19), (28, 'Na Pista', 20), (29, 'Axé Bahia 2001', 21), (30, 'BBC Sessions [Disc 1] [Live]', 22), (31, 'B

# **Above (SQLDatabase wrapper) and below (SQLDatabaseToolkit (tools for agents)**
---
# **1. SQLDatabase (the wrapper)**

`SQLDatabase` is a **low-level utility** that connects Python/LangChain to your SQL database.
It knows:

* how to open the database
* how to run SQL queries
* how to fetch results
* how to inspect tables and schema

Use this if you want to run SQL manually:

```python
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db.run("SELECT * FROM Artist LIMIT 5")
```

## **Note:** You (as the programmer) write SQL — the model is **not** involved in this case.

---

# **2. SQLDatabaseToolkit (tools for agents)**
A *toolkit* that converts the SQLDatabase into **tools that an AI agent can use**.

It creates 4 tools:

* **sql_db_query** → run SQL queries
* **sql_db_schema** → see schema
* **sql_db_list_tables** → list tables
* **sql_db_query_checker** → check SQL correctness


*“It's like a toolbox given to the AI agent so it can work independently.”*

The agent doesn’t know SQL.
The toolkit gives it instructions, guardrails, and tools.

---

# Why do we need *both*?

### 🔹 **SQLDatabase**

Used by **you** (the developer).
This is how you interact directly with SQLite.

### 🔹 **SQLDatabaseToolkit**

Used by **AI agents** (LangChain agents).
This allows the agent to:

* think about the user question
* understand tables
* decide which tool to call
* generate SQL
* validate SQL
* execute SQL via `sql_db_query`

### **Without SQLDatabase**

There is nothing for the toolkit to wrap.
The agent cannot access any data.

### **Without the Toolkit**

You can run SQL manually,
but the **AI assistant cannot independently query** the database.

---
# Conclusion:
* **SQLDatabase** → Direct SQL connection, used by your Python code
* **Toolkit** → Converts that connection into AI-usable tools
* **Agent** → Uses those tools to answer questions automatically

Together, they create a **fully autonomous SQL agent**.

## Reference Link: SQLDatabaseToolkit

- https://docs.langchain.com/oss/python/integrations/tools/sql_database

In [33]:
# from langchain_community.utilities.sql_database import SQLDatabase

# from sqlalchemy import create_engine

# from sqlalchemy.pool import StaticPool


# **SQLDatabase Kit is not compatible with langchain version we are using**


## So, we are using create_sql_agent from langchain community toolkits to build our SQL agent.

In [34]:
# !pip install -qU langchain langchain-openai langchain-community sqlalchemy

In [35]:
#create_sql_agent - > to build SQL Agent with our LLM and DB

from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI

In [36]:
# # downgrading the installation as some features are removed in the latest langchain version

# !pip install langchain==0.3.4
# !pip install langchain-community==0.3.3
# !pip install langchain-openai==0.2.2



In [37]:
db

In [38]:
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7b991e18d790>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7b991e18d670>, root_client=<openai.OpenAI object at 0x7b991e341580>, root_async_client=<openai.AsyncOpenAI object at 0x7b991e8c4e00>, model_name='gpt-4.1', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [39]:
# Import the NEW SQL Agent Function

from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentExecutor # to execute agent's plans

# **Creating SQL Agent**

create_sql_agent builds an agent that:

- understands your DB schema

- plans how to answer the question

- writes SQL

- executes SQL

- returns answers

agent_type="openai-tools" → newest method (OpenAI function calling → stable + accurate query generation)

verbose=True → shows internal steps like SQL queries

In [47]:
# create the sql agent

agent = create_sql_agent(
    llm=model,
    db=db,
    agent_type="openai-tools",   # NEW STYLE (no more react agent) - much better also
    verbose=True
)


In [48]:
# run the agent

agent.invoke({"input": "How many tracks are in the Chinook database?"})


# The agent now:

# Reads your question -> Inspects schema ->  Generates SQL query -> Runs SQL query ->
# Returns the answer in plain English ->  Displays reasoning in verbose mode

#This is the real power of SQL Agents.”




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Track'}`



CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the W

{'input': 'How many tracks are in the Chinook database?',
 'output': 'There are 3,503 tracks in the Chinook database.'}

In [42]:
agent.invoke({"input": "List 5 customers from India."})




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	S

{'input': 'List 5 customers from India.',
 'output': 'Here are customers from India (up to 5):\n\n1. Manoj Pareek - manoj.pareek@rediff.com\n2. Puja Srivastava - puja_srivastava@yahoo.in\n\nOnly two customers from India are present in the database.'}

# **VVI to read and understand**
---

# ✅ **Old Approach (SQLDatabaseToolkit)**

**How it worked:**

* You manually fetched **multiple tools**:

  * `sql_db_query`
  * `sql_db_schema`
  * `sql_db_list_tables`
  * `sql_db_query_checker`
* Then you manually created an **agent** that used these tools.
* More steps, more wiring, more chances for version conflicts.

**Mental model:**
You are giving the model a **toolbox**, and saying:
“Here are 4 tools. Use them wisely.”

---

# ✅ **New Approach (create_sql_agent)**

**How it works now:**

* LangChain automatically bundles **all SQL tools internally**.
* You only pass:

  * the LLM
  * the database
* It returns a fully-configured agent ready to answer SQL questions.

**Mental model:**
You say,
“Build me a full SQL analyst.”
LangChain automatically gives it the tools it needs.

---

# ⭐ **In one line:**

**Old approach = manually wiring individual SQL tools.**
**New approach = one function builds a complete SQL agent for you.**

#**So, now**

✔ No SQLDatabaseToolkit

It is deprecated in new versions.

✔ No create_react_agent

React agents have been replaced with OpenAI Tools / Function Agents.

✔ You can now use the agent directly

No need to fetch tools separately, the SQL agent internally handles:

query execution

schema inspection

result interpretation


#**Modifications:**


-- personal Database

-- UI building - interactive

-- use Langsmith, Langgraph

-- multiple databases